# Recording Experiment Workflow Results

While running an experiment workflow one would like to keep a record of what took place -- a kind of digital lab book. The LabOne Q Applications Library provides logbooks for just this task.

Each workflow run creates its own logbook. The logbook records the tasks being run and may also be used to store additional data such as device settings, LabOne Q experiments, qubits, and the results of experiments and analyses.

Logbooks need to be stored somewhere, and within the Applications Library, this place is called a logbook store.

Currently the Applications Library supports two kinds of stores:

* `FolderStore`
* `LoggingStore`

The `FolderStore` writes logbooks to a folder on disk. It is used to keep a permanent record of the experiment workflow.

The `LoggingStore` logs what is happening using Python's logging. It provides a quick overview of the steps performed by a workflow.

We'll look at each of these in more detail shortly, but first let us set up a quantum platform to run some experiments on so we have something to record.

## Setting up a quantum platform

Build your LabOne Q `DeviceSetup`, qubits and `Session` as normal. Here we import a demonstration tunable transmon quantum platform from the library and the amplitude Rabi experiment:

In [ ]:
import numpy as np
from laboneq.simple import *

from laboneq_applications.experiments import amplitude_rabi
from laboneq_applications.qpu_types.tunable_transmon import demo_platform

In [ ]:
# Create a demonstration QuantumPlatform for a tunable-transmon QPU:
qt_platform = demo_platform(n_qubits=6)

# The platform contains a setup, which is an ordinary LabOne Q DeviceSetup:
setup = qt_platform.setup

# And a tunable-transmon QPU:
qpu = qt_platform.qpu

# Inside the QPU, we have qubits, which is a list of six LabOne Q Application
# Library TunableTransmonQubit qubits:
qubits = qpu.qubits

In [ ]:
session = Session(setup)
session.connect(do_emulation=True)

## The LoggingStore

When you import the `laboneq_applications` library it automatically creates a default `LoggingStore` for you. This logging store is used whenever a workflow is executed and logs information about:

* the start and end of workflows
* the start and end of tasks
* any errors that occur
* comments (adhoc messages from tasks, more on these later)
* any data files that would be saved if a folder store was in use (more on these later too) 

These logs don't save anything on disk, but they will allow us to see what events are recorded and what would be saved if we did a have a folder store active.

### An example of logging

Let's run the amplitude Rabi experiment and take a look:

In [ ]:
amplitudes = np.linspace(0.0, 0.9, 10)
options = amplitude_rabi.experiment_workflow.options()
options.count(10)
options.averaging_mode("cyclic")
rabi_tb = amplitude_rabi.experiment_workflow(
    session,
    qpu,
    qubits[0],
    amplitudes,
    options=options,
)

The workflow has not yet been executed, but when you run the next cell, you should see messages like:

```
──────────────────────────────────────────────────────────────────────────────
 Workflow 'amplitude_rabi': execution started
────────────────────────────────────────────────────────────────────────────── 
```

appear in the logs beneath the cell.

In [ ]:
result = rabi_tb.run()

And that's all there is to the basic logging functionality.

### Advanced logging uses

If you need to create a logging store of your own you can do so as follows:

In [ ]:
from laboneq_applications.logbook import LoggingStore

logging_store = LoggingStore()

The logging store created above won't be active unless you run:

In [ ]:
logging_store.activate()

And you deactivate it with:

In [ ]:
logging_store.deactivate()

You can access the default logging store by importing it from `laboneq_applications.logbook`:

In [ ]:
from laboneq_applications.logbook import DEFAULT_LOGGING_STORE

DEFAULT_LOGGING_STORE

## The FolderStore

### Using the folder store

The `FolderStore` saves workflow results on disk and is likely the most important logbook store you'll use.

You can import it as follows:

In [ ]:
from laboneq_applications.logbook import FolderStore

To create a folder store you'll need to pick a folder on disk to store logbooks in. Here we select `./experiment_store` as the folder name but you should pick your own.

Each logbook created by a workflow will have its own sub-folder. The sub-folder name will start with a timestamp, followed by the name of the workflow, for example `20240728T175500-amplitude-rabi/`. If necessary, a unique count will be added at the end to make the sub-folder name unique.

The timestamps are in UTC, so they might be offset from your local time, but will be meaningful to users in other timezones and will remain correctly ordered when changing to or from daylight savings.

The folder store will need to be activated before workflows will use it automatically.

In [ ]:
folder_store = FolderStore("./experiment_store")
folder_store.activate()

Now let's run the amplitude Rabi workflow. As before we'll see the task events being logged. Afterwards we'll explore the folder to see what has been written to disk.

In [ ]:
result = rabi_tb.run()

If you no longer wish to automatically store workflow results in the folder store, you can deactivate it with:

In [ ]:
folder_store.deactivate()

### Exploring what was written to disk

Here we will use Python's `pathlib` functionality to explore what has been written to disk, but you can also use whatever ordinary tools you prefer (terminal, file navigator).

In [ ]:
import json
from pathlib import Path

Remember that above we requested that the folder store use a folder named `experiment_store`. Let's list the logbooks that were created in that folder:

In [ ]:
store_folder = Path("experiment_store")

amplitude_rabi_folders = sorted(store_folder.glob("*/*-amplitude-rabi"))

Our amplitude Rabi experiment is the most recent one run, so let's look at the files within the most recent folder. Note that the logbook folder names start with a timestamp followed by the name of the workflow, which allows us to easily order them by time and to find the workflow we're looking for:

In [ ]:
amplitude_rabi_folder = amplitude_rabi_folders[-1]

amplitude_rabi_files = sorted(
    amplitude_rabi_folder.iterdir()
)
amplitude_rabi_files

Let us look at the file `log.jsonl`. This is the log of what took place. The log is stored in a format called "JSONL" which means each line of the log is a simple Python dictionary stored as JSON. Larger objects and certain types of data are stored as separate files in a subfolder called `obj` or, for some important data, in the same folder.

Let's open the file and list the logs:

In [ ]:
experiment_log = amplitude_rabi_folder / "log.jsonl"
logs = [
    json.loads(line) for line in experiment_log.read_text().splitlines()
]
logs

In the remaining sections we'll look at how to write adhoc comments into the logs and how to save data files to disk.

The timestamp of the start time of the workflow execution and the name(s) of the currently executed workflow(s) (if the task was executed from a workflow) can be obtained from within a task. If the task was not called from within a workflow execution context, the timestamp will be None and the workflow names will be an empty list. Timestamp and the first of the workflow names are also part of the folder path in case a folder logger is used. Here is an example of a task which reads the outermost workflow's name and the timestamp:

In [ ]:
from laboneq_applications.workflow import (
    execution_info,
    task,
    workflow,
)


@task
def folder_logger_timestamp_and_workflow_name():
    info = execution_info()  # Returns a WorkflowExecutionInfoView object
    return (info.workflows[0], info.start_time)


@workflow
def timestamp_and_name_workflow():
    folder_logger_timestamp_and_workflow_name()


wf = timestamp_and_name_workflow()
result = wf.run()

print(result.tasks["folder_logger_timestamp_and_workflow_name"].output)

The output of `WorkflowExecutionInfoView.workflows` is a list, where the outermost workflow is the first element and the innermost workflow is the last element. The output of `WorkflowExecutionInfoView.start_time` is a `datetime.datetime` object, which is used for creating the folder logger's data folder in the format `YYYYMMDDTHHMMSS` (using `strftime("%Y%m%dT%H%M%S")`) after conversion from UTC to local time.

## Logging comments from within tasks

Logbooks allow tasks to add their own messages to the logbook as comments.

This is done by calling the `comment(...)` function within a task.

We'll work through an example below:

In [ ]:
from laboneq_applications.workflow import comment, task, workflow

Let's write a small workflow and a tiny task that just writes a comment to the logbook:

In [ ]:
@task
def log_a_comment(msg):
    comment(msg)


@workflow
def demo_comments():
    log_a_comment("Activating multi-state discrimination! <sirens blare>")
    log_a_comment("Analysis successful! <cheers>")

Now when we run the workflow we'll see the comments appear in the logs:

In [ ]:
wf = demo_comments()
result = wf.run()

Above you should see the two comments. They look like this:
```
Comment: Activating multi-state discrimination! <sirens blare>
...
Comment: Analysis successful! <cheers>
```

In addition to `comment(...)`, the logbook supports a function `log(level: int, message: str, *args: object)` which logs a message at the specified logging level similar to Python's `logging` module. This additional function is useful for logging messages that are not regular user comments, but allow tasks to give feedback about issues which are still important to record.

## Store data from within tasks

Logbooks also allow files to be saved to disk using the function `save_artifact`.

Here we will create a figure with matplotlib and save it to disk. The folder store will automatically save it as a PNG.

The kinds of objects the folder store can currently save are:

* Python strings (saved as a text file)
* Python bytes (saved as raw data)
* Pydantic models (saved as JSON)
* PIL images (saved as PNGs by default)
* Matplotlib figures (saved as PNGs by default)
* Numpy arrays (saved as Numpy data files)

Support for more kinds of objects coming soon (e.g. `DeviceSetup`, `Experiment`).

In [ ]:
import PIL
from matplotlib import pyplot as plt

from laboneq_applications.workflow import save_artifact

Let's write a small workflow that plots the sine function and saves the plot using `save_artifact`:

In [ ]:
@task
def sine_plot():
    fig = plt.figure()
    plt.title("A sine wave")
    x = np.linspace(0, 2 * np.pi, 100)
    y = np.sin(x)
    plt.plot(x, y)

    save_artifact("Sine Plot", fig)


@workflow
def demo_saving():
    sine_plot()

Since we deactivated the folder store, let's activate it again now:

In [ ]:
folder_store.activate()

And run our workflow:

In [ ]:
wf = demo_saving()
result = wf.run()

You can see in the logs that an artifact was created:
```
Artifact: 'Sine Plot' of type 'Figure' logged
```
Now let's load the image from disk.

First we need to find the logbook folder created for our workflow:

In [ ]:
demo_saving_folders = sorted(store_folder.glob("*/*-demo-saving"))
demo_saving_folder = demo_saving_folders[-1]
demo_saving_folder

And let's list its contents:

In [ ]:
sorted(demo_saving_folder.iterdir())

And finally let's load the saved image using PIL:

In [ ]:
PIL.Image.open(demo_saving_folder / "Sine Plot.png")

Saving an object also generates an entry in the folder store log.

We can view it by opening the log:

In [ ]:
experiment_log = demo_saving_folder / "log.jsonl"
logs = [
    json.loads(line) for line in experiment_log.read_text().splitlines()
]
logs

As you can see above the log records the name (`artifact_name`) and type (`artifact_type`) of the object saved, and the name of the file it was written to (`artifact_files`)

Saving an artifact might potentially write multiple files to disk.

The `artifact_metadata` contains additional user supplied information about the object saved, while `artifact_options` provide initial information on how to save the object. For example, we could have elected to save the figure in another file format. We'll see how to use both next.

### Specifying metadata and options when saving

Let's again make a small workflow that saves a plot, but this time we'll add some options and metadata.

In [ ]:
@task
def sine_plot_with_options():
    fig = plt.figure()
    plt.title("A sine wave")
    x = np.linspace(0, 2 * np.pi, 100)
    y = np.sin(x)
    plt.plot(x, y)
    [ax] = fig.get_axes()

    save_artifact(
        "Sine Plot",
        fig,
        metadata={
            "title": ax.get_title(),
        },
        options={
            "format": "jpg",
        },
    )


@workflow
def demo_saving_with_options():
    sine_plot_with_options()

And run the workflow to save the plot:

In [ ]:
wf = demo_saving_with_options()
result = wf.run()

Again we open the workflow folder and load the saved image:

In [ ]:
demo_saving_with_options_folders = sorted(
    store_folder.glob("*/*-demo-saving-with-options")
)
demo_saving_with_options_folder = demo_saving_with_options_folders[-1]
demo_saving_with_options_folder

In [ ]:
sorted(demo_saving_with_options_folder.iterdir())

Now when we load the image it is very slightly blurry, because it was saved as a JPEG which uses lossy compression:

In [ ]:
PIL.Image.open(demo_saving_with_options_folder / "Sine Plot.jpg")

And if we view the logs we can see that the title was recorded in the `artifact_metadata`:

In [ ]:
experiment_log = demo_saving_with_options_folder / "log.jsonl"
logs = [
    json.loads(line) for line in experiment_log.read_text().splitlines()
]
logs

The supported options for saving artifacts depend on the type of artifact. For our matplotlib figure example, the options are forwarded to `matplotlib.pyplot.savefig` and are documented in the [Matplotlib documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html), with the following changes to the default values:

* `format` is set to "png" by default
* `bbox_inches` is set to "tight" by default

In the same way, the options for a `PIL.Image.Image` are forwarded to `PIL.Image.Image.save` and are documented in the [Pillow documentation](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.save) with the format defaulting to "PNG". For a `numpy.ndarray` the options are forwarded to `numpy.save` and are documented in the [Numpy documentation](https://numpy.org/doc/stable/reference/generated/numpy.save.html) with `allow_pickle` set to `False` by default.

We're done!